In [12]:
##PREDICCIONES
import pandas as pd

In [13]:
df = pd.read_csv("inputs/data_predict.csv", header=0, delimiter=';')

/var/folders/jy/x2pqrfp13zq3nj0tq7p5wp400000gn/T/ipykernel_7998/3244851366.py:1: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("inputs/data_predict.csv", header=0, delimiter=';')


In [14]:
##formateo columnas objects que son numerics o date
df["ApplicationDate"] = pd.to_datetime(df["ApplicationDate"], format='mixed')
df["MonthlyIncome"] = pd.to_numeric(df['MonthlyIncome'], errors='coerce')
df["TotalDebtToIncomeRatio"] = pd.to_numeric(df['TotalDebtToIncomeRatio'], errors='coerce')
df["Gender"] = df["Gender"].replace({"F":0, "M":1})

/var/folders/jy/x2pqrfp13zq3nj0tq7p5wp400000gn/T/ipykernel_7998/1081767790.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["Gender"] = df["Gender"].replace({"F":0, "M":1})


In [15]:
df = df.drop(columns=["MonthlyIncome","LoanApproved"])


In [16]:
df = df.drop(["Gender","EmploymentStatus", "EducationLevel", "BankruptcyHistory", "PreviousLoanDefaults","ApplicationDate","MaritalStatus","HomeOwnershipStatus","MonthlyDebtPayments","LoanPurpose","RiskScore"], axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20002 entries, 0 to 20001
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Age                         20002 non-null  int64  
 1   AnnualIncome                20002 non-null  int64  
 2   CreditScore                 20002 non-null  int64  
 3   Experience                  20002 non-null  int64  
 4   LoanAmount                  20002 non-null  int64  
 5   LoanDuration                20002 non-null  int64  
 6   WithChilds                  20002 non-null  int64  
 7   NumberOfDependents          20002 non-null  int64  
 8   CreditCardUtilizationRate   20002 non-null  float64
 9   NumberOfOpenCreditLines     20002 non-null  int64  
 10  NumberOfCreditInquiries     20002 non-null  int64  
 11  DebtToIncomeRatio           20002 non-null  float64
 12  PaymentHistory              20002 non-null  int64  
 13  LengthOfCreditHistory       200

In [ ]:
orden = ["TotalDebtToIncomeRatio","AnnualIncome","BankruptcyHistory","LoanDuration","PreviousLoanDefaults"]

In [17]:
import joblib

In [18]:
modelo_xgboost = joblib.load("inputs/models/xgboos_bestmodel.joblib")

In [19]:
prediction = modelo_xgboost.predict(df)

ValueError: feature_names mismatch: ['TotalDebtToIncomeRatio', 'AnnualIncome', 'BankruptcyHistory_0', 'BankruptcyHistory_1', 'LoanDuration', 'PreviousLoanDefaults_0', 'PreviousLoanDefaults_1'] ['Age', 'AnnualIncome', 'CreditScore', 'Experience', 'LoanAmount', 'LoanDuration', 'WithChilds', 'NumberOfDependents', 'CreditCardUtilizationRate', 'NumberOfOpenCreditLines', 'NumberOfCreditInquiries', 'DebtToIncomeRatio', 'PaymentHistory', 'LengthOfCreditHistory', 'SavingsAccountBalance', 'CheckingAccountBalance', 'TotalAssets', 'TotalLiabilities', 'UtilityBillsPaymentHistory', 'JobTenure', 'NetWorth', 'BaseInterestRate', 'InterestRate', 'MonthlyLoanPayment', 'TotalDebtToIncomeRatio']
expected PreviousLoanDefaults_1, BankruptcyHistory_1, BankruptcyHistory_0, PreviousLoanDefaults_0 in input data
training data did not have the following fields: BaseInterestRate, DebtToIncomeRatio, WithChilds, CheckingAccountBalance, NumberOfDependents, JobTenure, MonthlyLoanPayment, LengthOfCreditHistory, Interest